## Import the necessary libraries

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Model

In [ ]:
# Fixed Model
class Value:
    # Telling that a input data is must to initiate the class
    # _children is to store before steps or values and its a tuple
    # Eg of _children [ for equation a*b+c Step1 is a*b stores in d and Step 2 is d + c then e here _prev(e) is d and c. _prev(d) is a and b ]
    # _op is to store the operation symbol so that it can be easy to understand in future in the visualization
    def __init__(self, data, _children = (), _op='', label = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        # _prev is to store childen as a set so that order doesn't matter
        self._prev = set(_children)
        self._op = _op
        self.label = label



    # Gives a more readable data instead of very small values or Giving only the data we enter
    def __repr__(self):
        return f"Value(data={self.data})"

    # Can be able to add two number
    def __add__(self,other):
        other = other if isinstance(other, Value) else Value(other) # this is for if a + 2 works but 2+a does not work as int adding to data so we are changing it to int to value object or data
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
          self.grad += 1.0*out.grad
          other.grad += 1.0*out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other): # other * self
      return self*other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __neg__(self): # -self
      return self * -1

    def __sub__(self, other): # self - other
      return self + (-other)

    def __radd__(self, other): # other + self
      return self + other

    # Can be able to multiply two numbers
    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other) # this is for if a * 2 works but 2 * a does not work as int adding to data so we are changing it to int to value object or data
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
          self.grad += other.data * out.grad
          other.grad += self.data*out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
      assert isinstance(other, (int, float)), "only supporting int/float powers for now"
      out = Value(self.data**other, (self,), f'**{other}')

      def _backward():
          self.grad += other * (self.data ** (other - 1)) * out.grad
      out._backward = _backward

      return out

    def tanh(self):
      x = self.data
      t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
      out = Value(t, (self,), 'tanh')
      def _backward():
        self.grad += (1 - t**2) * out.grad
      out._backward = _backward
      return out

    def exp(self):
      x = self.data
      out = Value(math.exp(x), (self, ), 'exp')

      def _backward():
        self.grad += out.data * out.grad
      out._backward = _backward

      return out

    def backward(self):
      # topological order code
      topo = []
      visited = set()
      def build_topo(v):
        if v not in visited:
          visited.add(v)
          for child in v._prev:
            build_topo(child)
          topo.append(v)
      build_topo(self)
      topo

      # initialize the first node with 1.0
      self.grad = 1.0

      for node in reversed(topo):
        node._backward()




In [ ]:
class Neuron:

  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b) # we use zip to align the data elements to target value like for ech input align w and b for that value generated
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]


class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()] # Collect parameters from each layer


In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
ypred = [n(x) for x in xs]
ypred

In [ ]:
# We first have to forward pass to build the network, then backward pass with
# loss then we can update or change the leaf node weights to tune the model
# The 100 is the epoch or no.of time you want to train and finetune the model
for k in range(100):

  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

  # backward pass
  for p in n.parameters():
    # we reset the grad values to '0' as in the Value model has '+=' but we want to replace the values so initialize it again with '0'
    p.grad = 0.0
  loss.backward()

  # update
  for p in n.parameters():
    # the -0.1 is the learning rate which means how much tweak you want to make to leaf node weights.
    p.data += -0.1 * p.grad

  print(k, loss.data)